In [95]:
#importer les packages necessaire
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [96]:
#importer les packages de machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, classification_report, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [97]:
#Chargement du DF
df = pd.read_excel('Copie de Coeur.xlsx')

In [98]:
#Travailler avec une copie du dataset pour éviter de le rechager en cas d'êrreur
df = df.copy()

In [99]:
#Afficher les informations relatives au DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [100]:
#Standardisation des variables quantitatives
for col in df.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df[col] = df[col]/df[col].max()


In [101]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,homme,AA,0.70,0.479270,0.0,Normal,0.851485,Non,0.000000,Ascendant,0
1,0.636364,femme,DNA,0.80,0.298507,0.0,Normal,0.772277,Non,0.161290,Plat,1
2,0.480519,homme,AA,0.65,0.469320,0.0,ST,0.485149,Non,0.000000,Ascendant,0
3,0.623377,femme,ASY,0.69,0.354892,0.0,Normal,0.534653,Oui,0.241935,Plat,1
4,0.701299,homme,DNA,0.75,0.323383,0.0,Normal,0.603960,Non,0.000000,Ascendant,0


In [102]:
#Encodage des variables qualitatives
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category').cat.codes

In [103]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2,1
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2,1
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0,0


In [104]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [105]:
#preparation des donnees de test et des donnees d'entrainememnt
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)

In [106]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [107]:
#Création d'un objet lr de la classe LogisticRegression
lr = LogisticRegression(solver ='newton-cg', random_state =1)

In [108]:
#Apprentissage du modèle
model = lr.fit(x_train, y_train)

In [109]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model.predict_proba(x_test)
predict_proba[:5,:]

array([[0.39560595, 0.60439405],
       [0.18843348, 0.81156652],
       [0.04970935, 0.95029065],
       [0.09007617, 0.90992383],
       [0.28158313, 0.71841687]])

In [110]:
#Application du modèle au données de test
y_pred = model.predict(x_test)
y_pred[:5]

array([1, 1, 1, 1, 1])

In [111]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[ 66,   8],
       [  6, 104]])

In [112]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9239130434782609

In [113]:
#Sensibilité
recall = recall_score(y_test, y_pred)
recall

0.9454545454545454

In [114]:
#Précision
precision = precision_score(y_test, y_pred)
precision

0.9285714285714286

In [115]:
#plus de détails avec classification report
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.92      0.89      0.90        74
           1       0.93      0.95      0.94       110

    accuracy                           0.92       184
   macro avg       0.92      0.92      0.92       184
weighted avg       0.92      0.92      0.92       184



In [116]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [117]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model.score(x_train, y_train)

0.8433242506811989

In [118]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model.score(x_test, y_test)

0.9239130434782609

# La partie de DecisionTree

In [119]:
df

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2,1
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2,1
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.584416,1,2,0.55,0.437811,0.0,1,0.653465,0,0.193548,2,1
914,0.883117,1,1,0.72,0.320066,1.0,1,0.698020,0,0.548387,2,1
915,0.740260,1,1,0.65,0.217247,0.0,1,0.569307,1,0.193548,2,1
916,0.740260,0,0,0.65,0.391376,0.0,0,0.861386,0,0.000000,2,1


In [120]:
#faire une copie du dataset de base pour avoir un autre sur lequel je pourais appliquer mon arbre de décision 
df_DT = df.copy()

In [121]:
df_DT.head(10)

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2,1
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2,1
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0,0
5,0.506494,1,3,0.60,0.562189,0.0,1,0.841584,0,0.000000,0,0
6,0.584416,0,0,0.65,0.393035,0.0,1,0.841584,0,0.000000,0,0
7,0.701299,1,0,0.55,0.344942,0.0,1,0.702970,0,0.000000,0,0
8,0.480519,1,1,0.70,0.343284,0.0,1,0.643564,1,0.241935,2,1
9,0.623377,0,0,0.60,0.470978,0.0,1,0.594059,0,0.000000,0,0


In [122]:
#Standardisation des variables quantitatives
for col in df_DT.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df_DT[col] = df_DT[col]/df_DT[col].max()

In [123]:
#Vérification des résultats
df_DT.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1.0,0.000000,0.70,0.479270,0.0,0.5,0.851485,0.0,0.000000,0.0,0
1,0.636364,0.0,1.000000,0.80,0.298507,0.0,0.5,0.772277,0.0,0.161290,1.0,1
2,0.480519,1.0,0.000000,0.65,0.469320,0.0,1.0,0.485149,0.0,0.000000,0.0,0
3,0.623377,0.0,0.333333,0.69,0.354892,0.0,0.5,0.534653,1.0,0.241935,1.0,1
4,0.701299,1.0,1.000000,0.75,0.323383,0.0,0.5,0.603960,0.0,0.000000,0.0,0


In [124]:
#Encodage des variables qualitatives dans une variable le
le = LabelEncoder()

In [125]:
#creer une variable df_encoded qui va stocker notre dataframe encoder
df_encoded=df.iloc[:,0:12]

In [126]:
#recodons par exemple le 'PENTE'
le.fit_transform(df_encoded["PENTE"])


array([0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2,
       0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0,
       2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0,
       2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2,
       2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0,
       2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2,
       2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2,

In [127]:
#Dans une fonction application cette modification à tout notre DataFrame
for i in df_encoded:
    df_encoded[i]=le.fit_transform(df_encoded[i])


In [128]:
df_encoded.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,12,1,0,41,147,0,1,98,0,10,0,0
1,21,0,3,55,40,0,1,82,0,20,2,1
2,9,1,0,31,141,0,2,25,0,10,0,0
3,20,0,1,39,72,0,1,34,1,25,2,1
4,26,1,3,49,53,0,1,48,0,10,0,0


In [129]:
# jeu de fonctionnalités
x=df_encoded.iloc[:,0:11]
#l'étiquettement de notre jeu
y=df_encoded.iloc[:,11]

In [130]:
#preparation des donnees de test et des donnees d'entrainememnt
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)

In [131]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [132]:
#Création d'un objet model de la classe DecisionTreeClassifier
model_DT=DecisionTreeClassifier(criterion='gini')

In [133]:
#Apprentissage du modèle
model_DT.fit(x_train,y_train)

DecisionTreeClassifier()

In [134]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model_DT.predict_proba(x_test)
predict_proba[:5,:]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [135]:
#Application du modèle au données de test
y_pred = model_DT.predict(x_test)
y_pred[:5]

array([1, 0, 1, 1, 0])

In [136]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[65,  9],
       [32, 78]])

In [137]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7771739130434783

In [138]:
#Sensibilité
recall_DT = recall_score(y_test, y_pred)
recall_DT

0.7090909090909091

In [139]:
#Précision
precision_DT = precision_score(y_test, y_pred)
precision_DT

0.896551724137931

In [140]:
#plus de détails avec classification report
cr_DT = classification_report(y_test, y_pred)
print(cr_DT)

              precision    recall  f1-score   support

           0       0.67      0.88      0.76        74
           1       0.90      0.71      0.79       110

    accuracy                           0.78       184
   macro avg       0.78      0.79      0.78       184
weighted avg       0.81      0.78      0.78       184



In [141]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model_DT.score(x_train, y_train)

1.0

In [142]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model_DT.score(x_test, y_test)

0.7771739130434783

# Comparaison des scores entre notre arbre de decision(DT) et notre Regression Logistic

In [143]:
#Comparaison au niveau des  Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
if model.score(x_train, y_train) > model_DT.score(x_train, y_train):
    print("Sur l'entrainement, le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision")
else:
    print("Sur l'entrainement, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision")

Sur l'entrainement, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision


In [144]:
#Comparaison au niveau des Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
if model.score(x_test, y_test) > model_DT.score(x_test, y_test):
    print("Sur l'entrainement,le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision")
else:
    print("Sur l'entrainement,le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision")

Sur l'entrainement,le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision


In [145]:
if model.score(x_train, y_train) < model_DT.score(x_train, y_train) and model.score(x_test, y_test) < model_DT.score(x_test, y_test):
    print("notre arbre de decision produit un bon résultat par raport à celle de la Régression Log")

notre arbre de decision produit un bon résultat par raport à celle de la Régression Log
